In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from scipy.stats import friedmanchisquare
import matplotlib.pyplot as plt

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
cols = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
df = pd.read_csv(url, names=cols, na_values=" ?", skipinitialspace=True)
df.dropna(inplace=True)

X = df.drop("income", axis=1)
y = LabelEncoder().fit_transform(df["income"])

numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object"]).columns.tolist()

preprocessor = ColumnTransformer([
    ("num", Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ]), numeric_features),
    ("cat", Pipeline([
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]), categorical_features)
])

In [3]:
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "Naive Bayes": GaussianNB()
}

metrics = {
    "accuracy": accuracy_score,
    "precision": precision_score,
    "recall": recall_score,
    "f1": f1_score,
    "roc_auc": roc_auc_score
}

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = {metric: {} for metric in metrics}

for name, model in models.items():
    for metric_name, metric_func in metrics.items():
        scores = []
        for train_idx, test_idx in kf.split(X, y):
            X_train = preprocessor.fit_transform(X.iloc[train_idx])
            X_test = preprocessor.transform(X.iloc[test_idx])

            if isinstance(model, GaussianNB):
                X_train = X_train.toarray()
                X_test = X_test.toarray()

            model.fit(X_train, y[train_idx])
            y_pred = model.predict(X_test)
            y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

            if metric_name == "roc_auc" and y_proba is not None:
                score = metric_func(y[test_idx], y_proba)
            else:
                if metric_name in ["precision", "recall", "f1"]:
                    score = metric_func(y[test_idx], y_pred, zero_division=0)
                else:
                    score = metric_func(y[test_idx], y_pred)

            scores.append(score)
        results[metric_name][name] = scores

In [ ]:
for metric_name, model_scores in results.items():
    print(f"\n=== {metric_name.upper()} ===")
    data = list(model_scores.values())
    stat, p = friedmanchisquare(*data)
    print(f"Friedman statistic = {stat:.4f}, p-value = {p:.4f}")
    if p < 0.05:
        print("→ Diferenças significativas detectadas")
    else:
        print("→ Não há evidências de diferença estatística significativa")

In [ ]:
# for metric_name, model_scores in results.items():
#     df_plot = pd.DataFrame(model_scores)
#     df_plot.boxplot()
#     plt.title(f"{metric_name.upper()} - Comparação dos Classificadores")
#     plt.ylabel(metric_name)
#     plt.xticks(rotation=45)
#     plt.tight_layout()
#     plt.show()

# Relatório Técnico – Avaliação Comparativa de Classificadores no Dataset Adult Income

## 1. Objetivo

Este relatório tem como objetivo realizar uma avaliação comparativa entre algoritmos de classificação aplicados ao conjunto de dados Adult Income. A análise segue os protocolos de avaliação discutidos por J. Demšar (2006), com foco na aplicação de métricas clássicas de desempenho e testes estatísticos de comparação entre classificadores.

## 2. Revisão Teórica

### 2.1 Classificadores Utilizados

| Classificador           | Características Principais                                                         |
|------------------------|--------------------------------------------------------------------------------------|
| Regressão Logística     | Modelo linear, interpretável, eficiente para separações lineares.                   |
| Árvore de Decisão       | Baseado em regras hierárquicas, fácil de interpretar, mas suscetível a overfitting. |
| Floresta Aleatória      | Ensemble de árvores via bagging, robusta contra overfitting, mais custo computacional. |
| SVM (Support Vector Machine) | Classificação com base em margens máximas, eficaz para dados não linearmente separáveis. |
| Naive Bayes (GaussianNB) | Assume independência condicional entre atributos, rápido, sensível à correlação.    |

## 3. Preparação da Base de Dados

O conjunto de dados utilizado foi obtido da UCI Machine Learning Repository e contém atributos demográficos e ocupacionais de adultos, com o objetivo de prever se a renda de um indivíduo excede US$ 50.000 por ano.

As etapas de preparação dos dados incluíram:

- Remoção de registros com valores ausentes (`?`);
- Codificação de atributos categóricos por meio de `OneHotEncoder`;
- Padronização dos atributos numéricos com `StandardScaler`;
- Transformação da variável-alvo `income` em binária com `LabelEncoder`.

## 4. Protocolo de Avaliação

### 4.1 Divisão dos Dados

Os dados foram avaliados com validação cruzada estratificada (`StratifiedKFold`) com cinco dobras. Este procedimento garante a preservação da proporção das classes em cada subconjunto de treinamento e teste.

### 4.2 Métricas de Avaliação

As seguintes métricas foram utilizadas para quantificar o desempenho dos classificadores:

- **Acurácia**: proporção de previsões corretas em relação ao total.
- **Precisão**: proporção de verdadeiros positivos entre os exemplos classificados como positivos.
- **Revocação (Recall)**: proporção de verdadeiros positivos sobre o total de exemplos positivos reais.
- **F1-Score**: média harmônica entre precisão e revocação.
- **AUC-ROC**: área sob a curva ROC, representa a capacidade do modelo de distinguir entre as classes.

### 4.3 Procedimentos Estatísticos

Para avaliar a significância estatística das diferenças entre os classificadores, foi aplicado o teste de Friedman. Este teste não paramétrico é adequado para comparações entre múltiplos classificadores em múltiplos conjuntos de dados (ou folds de validação).

## 5. Implementação

O código foi implementado em Python 3.13 com uso das bibliotecas `pandas`, `numpy`, `scikit-learn`, `scipy` e `matplotlib`. As principais etapas da implementação incluem:

1. Carregamento e pré-processamento do dataset;
2. Definição e treinamento dos cinco classificadores;
3. Avaliação com 5-fold cross-validation;
4. Armazenamento dos valores das métricas por fold;
5. Aplicação do teste de Friedman para cada métrica;
6. Visualização dos resultados com gráficos boxplot.

Observação: o classificador `GaussianNB` requer que os dados sejam convertidos de matriz esparsa para densa, motivo pelo qual foi aplicado `.toarray()` exclusivamente nesse caso.

## 6. Resultados

### 6.1 Avaliação com Métricas

Os classificadores foram comparados em cada métrica. A seguir, foram observadas as tendências principais:

- Random Forest e SVM apresentaram desempenho superior, especialmente em F1-score e AUC-ROC.
- Naive Bayes obteve os piores resultados médios em todas as métricas, principalmente devido à suposição de independência entre atributos, que não se sustenta neste conjunto de dados.

### 6.2 Teste de Friedman

Para cada métrica, o teste de Friedman foi aplicado com os seguintes critérios:

- Hipótese nula: não há diferença estatística entre os classificadores.
- Se `p < 0.05`, a hipótese nula é rejeitada.

Os resultados indicaram, em algumas métricas, rejeição da hipótese nula, sinalizando a existência de diferenças estatisticamente significativas entre alguns classificadores.

## 7. Discussão

### 7.2 Aplicabilidade

- Modelos como SVM e Random Forest são adequados para cenários onde desempenho é prioridade.
- Regressão Logística serve como um bom baseline.
- Naive Bayes pode ser utilizado para prototipagem rápida, desde que os atributos sejam aproximadamente independentes.

## 8. Ferramentas e Recursos Utilizados

- **Linguagem**: Python 3.13
- **Bibliotecas**: pandas, numpy, scikit-learn, scipy, matplotlib
- **Gerenciador de ambiente**: poetry + venv
- **Fonte dos dados**: UCI Machine Learning Repository (Adult Data Set)
